In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re 
import joblib
import string

In [2]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
fake = pd.read_csv('Datasets\Fake.csv')
true = pd.read_csv('Datasets\True.csv')

<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Irteja\AppData\Local\Temp\ipykernel_6428\3296890919.py:1: SyntaxWarning: invalid escape sequence '\F'
  fake = pd.read_csv('Datasets\Fake.csv')
C:\Users\Irteja\AppData\Local\Temp\ipykernel_6428\3296890919.py:2: SyntaxWarning: invalid escape sequence '\T'
  true = pd.read_csv('Datasets\True.csv')


In [4]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
fake['class'] = 0
true['class'] = 1

In [7]:
df = pd.concat([fake, true]).sample(frac=1, random_state=42)

In [8]:
# Combine title + text into one column
df['content'] = df['title'].fillna('') + " " + df['text'].fillna('') + " " + df['subject'].fillna('')

In [9]:
df.sample(10)

,title,text,subject,date,class,content
14609,Russia casts 10th U.N. veto on U.N. action on ...,UNITED NATIONS (Reuters) - Russia cast its 10t...,worldnews,"November 16, 2017",1,Russia casts 10th U.N. veto on U.N. action on ...
1067,Come Along On Cory Booker And Pals’ Field Tri...,"By now, the entire nation knows that the GOP i...",News,"June 21, 2017",0,Come Along On Cory Booker And Pals’ Field Tri...
21176,Thai university removes student leader for def...,BANGKOK (Reuters) - Thailand s prestigious Chu...,worldnews,"September 1, 2017",1,Thai university removes student leader for def...
12829,Zimbabwe judge frees ousted finance minister o...,HARARE (Reuters) - A Zimbabwean judge freed ou...,worldnews,"December 7, 2017",1,Zimbabwe judge frees ousted finance minister o...
5556,U.S. senators want say on any Trump Russia san...,WASHINGTON (Reuters) - (This February 8 story ...,politicsNews,"February 8, 2017",1,U.S. senators want say on any Trump Russia san...
1596,People Can Stop Giving Obama S*it About His S...,Our White House is currently occupied by someo...,News,"May 3, 2017",0,People Can Stop Giving Obama S*it About His S...
23408,BOILER ROOM – EP #48 – Agenda 2030 and Beyond ...,Tune in to the Alternate Current Radio Network...,Middle-east,"March 16, 2016",0,BOILER ROOM – EP #48 – Agenda 2030 and Beyond ...
7481,Malaysian PM says Trump appealed to Americans ...,KUALA LUMPUR (Reuters) - Malaysian Prime Minis...,politicsNews,"November 9, 2016",1,Malaysian PM says Trump appealed to Americans ...
18325,Russia's Putin says hasn't decided if he will ...,MOSCOW (Reuters) - Russian President Vladimir ...,worldnews,"October 4, 2017",1,Russia's Putin says hasn't decided if he will ...
5443,Kremlin denies allegations it violated U.S. mi...,MOSCOW (Reuters) - Russia is committed to hono...,politicsNews,"February 15, 2017",1,Kremlin denies allegations it violated U.S. mi...


In [10]:
df = df.drop(["title","subject","date","text"], axis = 1)

In [11]:
df.head()

,class,content
22216,0,Ben Stein Calls Out 9th Circuit Court: Committ...
4436,1,Trump drops Steve Bannon from National Securit...
1526,1,Puerto Rico expects U.S. to lift Jones Act shi...
1377,0,OOPS: Trump Just Accidentally Confirmed He Le...
8995,1,Donald Trump heads for Scotland to reopen a go...


In [12]:
df.reset_index(inplace=True)

In [13]:
# --- Text Cleaning ---
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub(r"[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\w*\d\w*", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [14]:
df['cleaned'] = df['content'].apply(clean_text)

In [15]:
df.sample(10)

,index,class,content,cleaned
7477,16690,1,Morocco arrests six suspected Islamic State mi...,morocco arrests six suspected islamic state mi...
25014,8357,1,Trump to give speech on illegal immigration on...,trump to give speech on illegal immigration on...
31037,7743,1,Polling safeguards stepped up after Trump's 'r...,polling safeguards stepped up after trumps rig...
5724,17311,0,REPORT: USDA GIVES $6.9 BILLION TO “INELIGIBLE...,report usda gives billion to “ineligible recip...
17693,20274,0,POLICE IN GERMANY BEGIN RAIDS On Homes Of Face...,police in germany begin raids on homes of face...
16913,12381,1,Algeria's ruling caste set on orderly successi...,algerias ruling caste set on orderly successio...
8330,7608,1,Gun control groups spend millions on state bal...,gun control groups spend millions on state bal...
29071,11518,1,"UNHCR says Australia abandoned refugees, must ...",unhcr says australia abandoned refugees must c...
13205,6343,0,Dozens Arrested During Massive Global Fossil ...,dozens arrested during massive global fossil f...
27580,12616,1,Swedish PM condemns attempted arson attack at ...,swedish pm condemns attempted arson attack at ...


In [16]:
x =df["cleaned"]
y=df["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.25, random_state=42)

In [17]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [18]:
nb = MultinomialNB()
nb.fit(xv_train, ytrain)

MultinomialNB()

In [19]:
y_pred = nb.predict(xv_test)
nb.score(xv_test,ytest)

0.9404899777282851

In [21]:
print("Accuracy:", accuracy_score(ytest, y_pred))
print("\nClassification Report:\n", classification_report(ytest, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(ytest, y_pred))

Accuracy: 0.9404899777282851

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      5863
           1       0.94      0.94      0.94      5362

    accuracy                           0.94     11225
   macro avg       0.94      0.94      0.94     11225
weighted avg       0.94      0.94      0.94     11225


Confusion Matrix:
 [[5525  338]
 [ 330 5032]]


In [25]:
joblib.dump(vectorizer, "vectorizer_nb.jb")
joblib.dump(nb, "nb_model.jb")


['nb_model.jb']